The homoskedasticity assumption, introduced in Chapter 3 for multiple regression, states that the variance of the unobserved error, u , conditional on the explanatory variables, is constant. Homoskedasticity fails whenever the variance of the unobserved factors changes across different segments of the population, where the segments are determined by the different values of the explanatory variables. For example, in a savings equation, heteroskedasticity is present if the variance of the unobserved factors affecting savings increases with income.

In Chapters 4 and 5, we saw that homoskedasticity is needed to justify the usual t tests, F tests, and confidence intervals for OLS estimation of the linear regression model, even with large sample sizes. In this chapter, we discuss the available remedies when heteroskedasticity occurs, and we also show how to test for its presence. We begin by briefly reviewing the consequences of heteroskedastic- ity for ordinary least squares estimation.

## 8.2 Heteroskedasticity-Robust Inference after OLS Estimation

Because testing hypotheses is such an important component of any econometric analysis and the usual OLS inference is generally faulty in the presence of heteroskedasticity, we must decide if we should entirely abandon OLS. Fortunately, OLS is still useful. In the last two decades, econometricians have learned how to adjust standard errors and t , F , and LM statistics so that they are valid in the presence of heteroske- dasticity of unknown form . This is very convenient because it means we can report new statistics that work regardless of the kind of heteroskedasticity present in the population. The methods in this section are known as heteroskedasticity-robust procedures because they are valid -at least in large samples- whether or not the errors have constant variance, and we do not need to know which is the case.

### Wooldridge example 8.2 Heteroskedasticity-Robust F Statistic

Using the data for the spring semester in GPA3, we first estimate the model with the usual OLS standard errors

In [6]:
library(foreign)
gpa3 <- read.dta("https://github.com/thousandoaks/Wooldridge/blob/master/gpa3.dta?raw=true")

# load packages (which need to be installed!)
library(lmtest); library(car)

# Estimate model (only for spring data)
reg <- lm(cumgpa~sat+hsperc+tothrs+female+black+white, 
                                     data=gpa3, subset=(spring==1))
# Usual SE:
coeftest(reg)




t test of coefficients:

               Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)  1.47006477  0.22980308  6.3971 4.942e-10 ***
sat          0.00114073  0.00017856  6.3885 5.197e-10 ***
hsperc      -0.00856636  0.00124042 -6.9060 2.275e-11 ***
tothrs       0.00250400  0.00073099  3.4255 0.0006847 ***
female       0.30343329  0.05902033  5.1412 4.497e-07 ***
black       -0.12828369  0.14737012 -0.8705 0.3846163    
white       -0.05872173  0.14098956 -0.4165 0.6772953    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


And then we do the same this time using heteroskedasticity-robust standard errors

In [5]:
# Refined White heteroscedasticity-robust SE:
coeftest(reg, vcov=hccm)


t test of coefficients:

               Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)  1.47006477  0.22938036  6.4089 4.611e-10 ***
sat          0.00114073  0.00019532  5.8402 1.169e-08 ***
hsperc      -0.00856636  0.00144359 -5.9341 6.963e-09 ***
tothrs       0.00250400  0.00074930  3.3418   0.00092 ***
female       0.30343329  0.06003964  5.0539 6.911e-07 ***
black       -0.12828369  0.12818828 -1.0007   0.31762    
white       -0.05872173  0.12043522 -0.4876   0.62615    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


Again, the differences between the usual standard errors and the heteroskedasticity-robust standard errors are not very big, and use of the robust t statistics does not change the statistical significance of any independent variable. This is an indication that heteroskedasticity might not be a big issue in this example.

Suppose we wish to test the null hypothesis that, after the other factors are controlled for, there are no differences in cumgpa by race. This is stated as $H_0:\beta_{black}=0,\beta_{white}=0$. The usual F statistic is easily obtained,

In [8]:
# F-Tests using different variance-covariance formulas:
myH0 <- c("black","white")
# Ususal VCOV
linearHypothesis(reg, myH0)


Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
361,79.36167,NA,NA,NA,NA
359,79.06233,2,0.2993375,0.6796042,0.5074684


If heteroskedasticity is present, this version of the test is invalid. The heteroskedasticity-robust version has no simple form, but it can be computed using certain statistical packages, refer to the following code. The value of the heteroskedasticity-robust F statistic turns out to be .75, which differs only slightly from the nonrobust version. The p -value for the robust test is .474, which is not close to standard significance levels. We fail to reject the null hypothesis using either test.

In [10]:
# Classical White VCOV
linearHypothesis(reg, myH0, vcov=hccm(reg,type="hc0"))

Res.Df,Df,F,Pr(>F)
361,NA,NA,NA
359,2,0.747797,0.4741443


## 8.3 Testing for Heteroskedasticity

The heteroskedasticity-robust standard errors provide a simple method for computing t statistics that are asymptotically t distributed whether or not heteroskedasticity is present. We have also seen that heteroskedasticity-robust F and LM statistics are available. Implementing these tests does not require knowing whether or not heteroskedasticity is present. Nevertheless, there are still some good reasons for having simple tests that can detect its presence. First, as we mentioned in the previous section, the usual t statistics have exact t distributions under the classical linear model assumptions. For this reason, many economists still prefer to see the usual OLS standard errors and test statistics reported, unless there is evidence of heteroskedasticity.

Second, if heteroskedasticity is present, the OLS estimator is no longer the best linear unbiased estimator. As we will see in Section 8-4, it is possible to obtain a better estimator than OLS when the form of heteroskedasticity is known.

#### The Breusch-Pagan Test for Heteroskedasticity:

Please refer to Wooldridge 2016 (section 8.3) for a detailed analysis of the test

### Wooldridge Example 8.4. Heteroskedasticity in Housing Price Equations

We use the data in HPRICE1 to test for heteroskedasticity in a simple housing price equation using the Breusch-Pagan test (refer to Wooldridge 2016, section 8.3 for more information). The estimated equation using the levels of all variables is:

In [13]:
library(foreign)
hprice1<-read.dta("https://github.com/thousandoaks/Wooldridge/blob/master/hprice1.dta?raw=true")

# Estimate model
reg <- lm(price~lotsize+sqrft+bdrms, data=hprice1)
summary(reg)

# Automatic BP test
library(lmtest)
bptest(reg)


Call:
lm(formula = price ~ lotsize + sqrft + bdrms, data = hprice1)

Residuals:
     Min       1Q   Median       3Q      Max 
-120.026  -38.530   -6.555   32.323  209.376 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.177e+01  2.948e+01  -0.739  0.46221    
lotsize      2.068e-03  6.421e-04   3.220  0.00182 ** 
sqrft        1.228e-01  1.324e-02   9.275 1.66e-14 ***
bdrms        1.385e+01  9.010e+00   1.537  0.12795    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 59.83 on 84 degrees of freedom
Multiple R-squared:  0.6724,	Adjusted R-squared:  0.6607 
F-statistic: 57.46 on 3 and 84 DF,  p-value: < 2.2e-16



	studentized Breusch-Pagan test

data:  reg
BP = 14.092, df = 3, p-value = 0.002782


The LM statistic is 14.09 which gives a p-value < .00278 (using the $\chi_3^2$ distribution), thus the usual standard errors reported in the previous result are not reliable.

In Chapter 6, we mentioned that one benefit of using the logarithmic functional form for the dependent variable is that heteroskedasticity is often reduced. In the current application, let us put price , lotsize , and sqrft in logarithmic form, so that the elasticities of price , with respect to lotsize and sqrft , are constant. The estimated equation is

In [19]:
library(foreign)
hprice1<-read.dta("https://github.com/thousandoaks/Wooldridge/blob/master/hprice1.dta?raw=true")

# Estimate model
reglog <- lm(log(price)~log(lotsize)+log(sqrft)+bdrms, data=hprice1)
summary(reglog)

#B-P test
bptest(reglog)


Call:
lm(formula = log(price) ~ log(lotsize) + log(sqrft) + bdrms, 
    data = hprice1)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.68422 -0.09178 -0.01584  0.11213  0.66899 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -1.29704    0.65128  -1.992   0.0497 *  
log(lotsize)  0.16797    0.03828   4.388 3.31e-05 ***
log(sqrft)    0.70023    0.09287   7.540 5.01e-11 ***
bdrms         0.03696    0.02753   1.342   0.1831    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.1846 on 84 degrees of freedom
Multiple R-squared:  0.643,	Adjusted R-squared:  0.6302 
F-statistic: 50.42 on 3 and 84 DF,  p-value: < 2.2e-16



	studentized Breusch-Pagan test

data:  reglog
BP = 4.2232, df = 3, p-value = 0.2383


Regressing the squared OLS residuals from this regression on log(lotsize), log(sqrft), and bdrms give an LM statistic of LM=4.22 (p-value=.239). Therefore, we fail to reject the null hypothesis of homoskedasticity in the model with the logarithmic functional forms. The occurrence of less heteroskedasticity with the dependent variable in logarithmic form has been noticed in many empirical applications.

## Summary

We began by reviewing the properties of ordinary least squares in the presence of heteroskedasticity. Heteroskedasticity does not cause bias or inconsistency in the OLS estimators, but the usual standard errors and test statistics are no longer valid. We showed how to compute heteroskedasticity-robust standard errors and t statistics, something that is routinely done by many regression packages. Most regression packages also compute a heteroskedasticity-robust F -type statistic.